<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Prepare-vvgish-input-manually" data-toc-modified-id="Prepare-vvgish-input-manually-0.0.1"><span class="toc-item-num">0.0.1&nbsp;&nbsp;</span>Prepare vvgish input manually</a></span></li><li><span><a href="#We-can-directly-go-from-path-of-mp3-file-to-input-vggish" data-toc-modified-id="We-can-directly-go-from-path-of-mp3-file-to-input-vggish-0.0.2"><span class="toc-item-num">0.0.2&nbsp;&nbsp;</span>We can directly go from path of mp3 file to input vggish</a></span></li></ul></li></ul></li><li><span><a href="#Index-FLow" data-toc-modified-id="Index-FLow-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Index FLow</a></span><ul class="toc-item"><li><span><a href="#Segmenting-the-data" data-toc-modified-id="Segmenting-the-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Segmenting the data</a></span></li><li><span><a href="#Passing-data-thorugh-the-vvgish" data-toc-modified-id="Passing-data-thorugh-the-vvgish-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Passing data thorugh the vvgish</a></span></li><li><span><a href="#Indexing" data-toc-modified-id="Indexing-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Indexing</a></span></li></ul></li><li><span><a href="#Query-Flow" data-toc-modified-id="Query-Flow-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Query Flow</a></span></li><li><span><a href="#Other-info" data-toc-modified-id="Other-info-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Other info</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Finding-matches-in-a-dataset" data-toc-modified-id="Finding-matches-in-a-dataset-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Finding matches in a dataset</a></span></li></ul></li></ul></li></ul></div>

In [277]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [278]:
from vggish import mel_features
from vggish import vggish_input
from vggish import vggish_slim
import executors

import numpy as np
import inspect

import jina
from jina import Document, DocumentArray

### Prepare vvgish input manually

In [279]:
import librosa

In [280]:
x_audio, sample_rate = librosa.load('data/sample.mp3')

/Users/davidbuchaca1/opt/anaconda3/lib/python3.8/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [281]:
log_mel_examples = vggish_input.waveform_to_examples(x_audio, sample_rate)

In [282]:
log_mel_examples.shape

(28, 96, 64)

### We can directly go from path of mp3 file to input vggish

In [283]:
log_mel_examples = vggish_input.mp3_to_examples('data/sample.mp3')

/Users/davidbuchaca1/opt/anaconda3/lib/python3.8/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [284]:
log_mel_examples.shape

(28, 96, 64)

In [285]:
log_mel_examples = vggish_input.wavfile_to_examples('data/Beethoven_1.wav')

In [286]:
log_mel_examples.shape, log_mel_examples.flatten().shape

((19, 96, 64), (116736,))

# Index FLow

## Segmenting the data

The segmenter in audio-example from jina examples does the following at segment time:

- uses `read_wav(path_to_wav)` reads a wav file and returns a numpy array (`data`) and a integer (`sample_rate`)

    - Here `read_wav` reads the data from a path, calling `soundfile.read` which returns a numpy array `data`.



Example for `data/Beethoven_1.wav`:

- data after `soundfile.read` has shape (806912,)

- `mel_data=wav2vel(data)` has shape (806912,)




In [287]:
import soundfile as sf

wav_data, sample_rate = sf.read('data/Beethoven_1.wav', dtype='int16')
print(wav_data.shape)
wav_data = np.mean(wav_data, axis=1)
data = wav_data / sample_rate
print(data.shape)

(806912, 2)
(806912,)


In [288]:
from vggish.vggish_input import waveform_to_examples

aux = waveform_to_examples(data, sample_rate)
print(aux.shape)

aux = waveform_to_examples(data, sample_rate).squeeze()
print(aux.shape)

(19, 96, 64)
(19, 96, 64)


In [289]:
segmenter = executors.VggishSegmenter()

In [290]:
print(inspect.getsource(segmenter.segment))

    def segment(self, docs, *args, **kwargs):
        for doc in docs:
            data, sample_rate = self.read_wav(doc.uri)
            mel_data = self.wav2mel(data, sample_rate)
            for idx, blob in enumerate(mel_data):
                #self.logger.debug(f'blob: {blob.shape}')
                doc.chunks.append(Document(offset=idx, weight=1.0, blob=blob))



Note that each chunk is a Document containing each element from mel_data.

In particular this will create as elements as mel_data.shape[0]

In [291]:
aux[0].shape

(96, 64)

##### Segmenting data

In [292]:
doc_1 = Document(uri = 'data/Beethoven_1.wav')
doc_2 = Document(uri = 'data/Beethoven_2.wav')

print(doc_1.uri, doc_2.uri)
darray = DocumentArray([doc_1, doc_2])
segmenter = executors.VggishSegmenter()

data/Beethoven_1.wav data/Beethoven_2.wav


Now we can see that a document contains nothing more than a uri

In [293]:
print(darray[0])

{'id': '973b30b6-c21e-11eb-8d75-787b8ab3f5de', 'uri': 'data/Beethoven_1.wav', 'mime_type': 'audio/x-wav'}


In [294]:
segmenter.read(darray)

After segmenting the document contains many chunks

In [295]:
darray[0]

Note that different documents can have a different number of chunks

In [296]:
len(darray[0].chunks),len(darray[1].chunks)

(19, 33)

##  Passing data thorugh the vvgish


Now we have each document with a bunch of chunks and we need to encode the data of each chunk as an embedding

In [297]:
darray[0].chunks

<jina.types.arrays.chunk.ChunkArray length=19 at 140544965935888>

In [298]:
from executors import VggishEncoder

In [424]:
encoder = VggishEncoder()

INFO:tensorflow:Restoring parameters from /Users/davidbuchaca1/Documents/git_stuff/neuralsearch/jina_2/audio-search_V1/models/vggish_model.ckpt


In [425]:
type(darray[0].chunks[0].embedding)

numpy.ndarray

#### Traversing over chunks

We can traverse over all chunks of all documents using `darray.traverse_flat(traversal_paths='c')` which will return a DocumentArray that contains 52 documents (19+33=52).

In [426]:
darray.traverse_flat(traversal_paths='c')

<jina.types.arrays.document.DocumentArray length=52 at 140544391462384>

In [302]:
embedding = encoder._encode(darray.traverse_flat(traversal_paths='c'))

After the embedding is generated we have a vector of 128 components for each chunk

In [303]:
print(inspect.getsource(encoder._encode))

    def _encode(self, docs: DocumentArray, *args, **kwargs):
        blobs = docs.get_attributes('blob')
        [embedding_batch] = self.sess.run([self.embedding_tensor],
                                          feed_dict={self.feature_tensor: blobs})
        result = self.post_processor.postprocess(embedding_batch)
        embedding_matrix = (np.float32(result) - 128.) / 128.
        return embedding_matrix



In [304]:
embedding.shape

(52, 128)

The function `_encode_and_update` precisely updates the document array
using each of the rows in the matrix generated by `_encode`

In [305]:
print(inspect.getsource(encoder._encode_and_update))

    def _encode_and_update(self, docs: DocumentArray, *args, **kwargs):
        embedding_matrix = self._encode(docs)
        for d,e in zip(docs, embedding_matrix):
            d.embedding = e



In [306]:
encoder._encode_and_update(darray.traverse_flat(traversal_paths='c'))

In [307]:
darray[0].chunks[0].embedding.shape

(128,)

## Indexing

We can create an `Indexer object` and pass the document array with the DocumentArray that contains the embeddings

In [308]:
indexer = executors.Indexer()

In [309]:
indexer.index(darray)

When the indexer is fitted (`.index()`) it stores the embeddings of the data

In [310]:
indexer._docs[0].chunks[0].embedding.shape

(128,)

In [311]:
#this is empty
type(indexer._embedding_matrix)

NoneType

In [316]:
type(indexer._darray_chunks)

NoneType

Note that now folder `workspace` does not exists

In [313]:
!ls workspace/

ls: workspace/: No such file or directory


after calling `indexer.close()` the `workspace` folder will be created

In [318]:
indexer.close()

In [320]:
!ls workspace/

docs.json


Then when we instanciate the indexer again it will contain the `_embemdding_matrix` which will be loaded from `workspace`

In [377]:
indexer = executors.Indexer()

[autoreload of executors failed: Traceback (most recent call last):
  File "/Users/davidbuchaca1/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/davidbuchaca1/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/davidbuchaca1/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/davidbuchaca1/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/Users/davidbuchaca1/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/davidbuchaca1/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line

In [378]:
indexer._embedding_matrix

array([[ 0.234375 , -0.421875 ,  0.0859375, ...,  0.4921875,  0.6640625,
        -0.046875 ],
       [ 0.2890625, -0.4296875,  0.0625   , ...,  0.6953125,  0.3515625,
        -0.921875 ],
       [ 0.2890625, -0.5703125,  0.15625  , ...,  0.5625   ,  0.8203125,
         0.       ],
       ...,
       [ 0.34375  , -0.796875 ,  0.       , ...,  0.1328125,  0.078125 ,
         0.4765625],
       [ 0.34375  , -0.796875 , -0.0078125, ...,  0.09375  ,  0.0703125,
         0.4453125],
       [ 0.3515625, -0.7890625, -0.015625 , ...,  0.0390625,  0.015625 ,
         0.4609375]], dtype=float32)

# Query Flow

Let us assume we have a query that is segmented and for each segment a chunk is created with an embedding

In [415]:
chunks = darray[0].chunks[2:4]
q = Document()
q.chunks = chunks

In [416]:
q

In [417]:
q_darray = DocumentArray([q])

Now it is time for the `Indexer` to find matches

In [418]:
indexer.search(q_darray, parameters={'top_k':10})

In [419]:
q_darray[0].matches

<jina.types.arrays.match.MatchArray length=2 at 140543733364144>

In [420]:
q_darray[0].matches[0].score.value

0.0

In [421]:
q_darray[0].matches[0].uri

'data/Beethoven_1.wav'

In [422]:
q_darray[0].matches[1].uri

'data/Beethoven_2.wav'

# Other info

Note that different wav files will be represented with different numpy arrays:

In [49]:
data_1 = vggish_input.wavfile_to_examples('data/Beethoven_1.wav')
data_2 = vggish_input.wavfile_to_examples('data/Beethoven_2.wav')
print(data_1.shape)
print(data_2.shape)

(19, 96, 64)
(33, 96, 64)


Now let's define a VggishEncoder and look at how to encode the data

In [30]:
encoder = VggishEncoder()

/Users/davidbuchaca1/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/Users/davidbuchaca1/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:329: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '


INFO:tensorflow:Restoring parameters from /Users/davidbuchaca1/Documents/git_stuff/neuralsearch/jina_2/audio-search_V1/models/vggish_model.ckpt


In [31]:
inspect.getsourcefile(encoder._encode)

'/Users/davidbuchaca1/Documents/git_stuff/neuralsearch/jina_2/audio-search_V1/executors.py'

We can take a look at how data is encoded

In [32]:
print(inspect.getsource(encoder._encode))

    def _encode(self, docs: DocumentArray, *args, **kwargs):
        blobs = docs.get_attributes('blob')
        [embedding_batch] = self.sess.run([self.embedding_tensor],
                                           feed_dict={self.feature_tensor: blobs})
        result = self.post_processor.postprocess(embedding_batch)
        embedding_matrix = (np.float32(result) - 128.) / 128.
        
        for d,e in zip(docs, embedding_matrix):
            d.embedding = e



In [33]:
encoder._encode

<bound method VggishEncoder._encode of <executors.VggishEncoder object at 0x7f90d1184f70>>

In [34]:
d1 = Document()
d2 = Document()

d1.blob = data_1
d2.blob = data_2

darray = DocumentArray([d1,d2])

In [36]:
d1.blob.shape, d2.blob.shape

((19, 96, 64), (33, 96, 64))

In [42]:
aux = darray.get_attributes('blob')

In [47]:
aux[0].shape

(19, 96, 64)

In [31]:
#encoder._encode(darray)

In [85]:
#Document(data_1)

In [357]:
np.min([2,3,4])

2

### Finding matches in a dataset 

We have seen that given an audio input we create a numpy array `(n, 96, 64)` where `n` depends on the input audio.

At index time we will index each segment to a vector `(n, 96, 64)` will be converted to



```
(0, 96, 64)   ->  (96, 64)  -> embedding
(1, 96, 64)   ->  (96, 64)  -> embedding
...
(n-1, 96, 64) ->  (96, 64)  -> embedding
```


Then given a query with shape `(n_q, 96,64)` we will create an embedding for each chunk in the query


```
(0, 96, 64)     ->  (96, 64)  -> embedding
(1, 96, 64)     ->  (96, 64)  -> embedding
...
(n_q-1, 96, 64) ->  (96, 64)  -> embedding
```

Afterwards we will find for each chunk embedding which is its closest match.

Given a chunk from the query $c_k$ we will compute

```
d(c_k, c) for each c 
```

Then we will get the 'hitted' parents

```
parent_id_1 : [0.1, 0.3]
parent_id_2 : [0.3, 0.4]
...
parent_id_N : [0.1]
```






In [18]:
print(inspect.getsource(segmenter.read))

    def read(self, docs, *args, **kwargs):
        for doc in docs:
            data, sample_rate = self.read_wav(doc.uri)
            mel_data = self.wav2mel(data, sample_rate)
            for idx, blob in enumerate(mel_data):
                # self.logger.debug(f'blob: {blob.shape}')
                doc.chunks.append(Document(offset=idx, weight=1.0, blob=blob))



In [19]:
data_1 = vggish_input.wavfile_to_examples('data/Beethoven_1.wav')
data_2 = vggish_input.wavfile_to_examples('data/Beethoven_2.wav')
print(data_1.shape)
print(data_2.shape)

(19, 96, 64)
(33, 96, 64)


In [20]:
segmenter = executors.VggishSegmenter()

In [21]:
data, sample_rate = segmenter.read_wav('data/Beethoven_1.wav')
data.shape

(806912,)

In [22]:
data, sample_rate = segmenter.read_wav('data/Beethoven_2.wav')
data.shape

(1409024,)